<a href="https://colab.research.google.com/github/Malerion/Archives-Python/blob/main/NLP_4_Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive


/content/drive/MyDrive


In [3]:
import zipfile

with zipfile.ZipFile('tweets.zip', 'r') as zip_ref:
    zip_ref.extractall('destination_folder')


In [62]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/destination_folder/train.csv')
df.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [77]:
#exercice 2
# Exclure les tweets neutres
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]

# Calculer le pourcentage de tweets positifs/négatifs
positive_percentage = (df_filtered['sentiment'] == 'positive').mean() * 100
negative_percentage = (df_filtered['sentiment'] == 'negative').mean() * 100

print(f"Pourcentage de tweets positifs : {positive_percentage:.2f}%")
print(f"Pourcentage de tweets négatifs : {negative_percentage:.2f}%")


Pourcentage de tweets positifs : 52.45%
Pourcentage de tweets négatifs : 47.55%


In [78]:
#exo 3
from sklearn.model_selection import train_test_split

# Copie des colonnes dans des Séries
X = df[df['sentiment'].isin(['positive', 'negative'])]['text']
y = df[df['sentiment'].isin(['positive', 'negative'])]['sentiment']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)


In [80]:
#exo 4
from sklearn.feature_extraction.text import CountVectorizer

# Création du modèle vectorizer et entraînement sur X_train
vectorizer = CountVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)

# Transformation de X_test en utilisant le vocab de X_train
X_test_CV = vectorizer.transform(X_test)

print("Dimensions de X_test_CV :", X_test_CV.shape)
print("Nombre d'éléments stockés dans la matrice creuse (stored elements) :", X_test_CV.nnz)


Dimensions de X_test_CV : (4091, 15806)
Nombre d'éléments stockés dans la matrice creuse (stored elements) : 44633


In [83]:
#ex 5
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings

# Ignorer les avertissements de convergence
warnings.filterwarnings("ignore", category=UserWarning)


classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_CV, y_train)

y_train_pred = classifier.predict(X_train_CV)
y_test_pred = classifier.predict(X_test_CV)

accuracy_train = accuracy_score(y_train, y_train_pred)
print("Précision sur les données d'entraînement :", round(accuracy_train, 3))

accuracy_test = accuracy_score(y_test, y_test_pred)
print("Précision sur les données de test :", round(accuracy_test, 3))



Précision sur les données d'entraînement : 0.966
Précision sur les données de test : 0.877


In [85]:
# on tente de comparer les prédictions avec les étiquettes réelles
misclassified_indices = (y_test != y_test_pred)

# pour obtnir les textes des tweets mal prédits
misclassified_tweets = X_test[misclassified_indices]
true_labels = y_test[misclassified_indices]
predicted_labels = y_test_pred[misclassified_indices]

# 10 tweets mal prédits
for i in range(10):
    print(f"Tweet: {misclassified_tweets.iloc[i]}")
    print(f"Étiquette réelle : {true_labels.iloc[i]}, Prédiction : {predicted_labels[i]}\n")


Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
Étiquette réelle : negative, Prédiction : positive

Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Étiquette réelle : negative, Prédiction : positive

Tweet: i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
Étiquette réelle : positive, Prédiction : negative

Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Étiquette réelle : negative, Prédiction : positive

Tweet: I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
Étiquette réelle : positive, Prédiction : negative

Tweet: wish I could feel no pain (8)  but it`s ok, at least they like Brazil!
Étiquette réelle : positive, Prédiction : negative

